_Note: This is a SQL scoped notebook, and therefore should be attached to a Serverless SQL warehouse for execution._  

In [0]:
DECLARE OR REPLACE VARIABLE catalog_use STRING DEFAULT 'fhir_workshop';
DECLARE OR REPLACE VARIABLE schema_use STRING DEFAULT REPLACE(SPLIT(current_user(), '@')[0], '.', '_');
DECLARE OR REPLACE VARIABLE resource_type STRING DEFAULT 'Encounter';
DECLARE OR REPLACE VARIABLE full_refresh BOOLEAN DEFAULT FALSE;

In [0]:
SET VARIABLE catalog_use = :catalog_use; 
SET VARIABLE schema_use = :schema_use;
SET VARIABLE resource_type = :resource_type; 
SET VARIABLE full_refresh = :full_refresh == 'true';

SELECT resource_type, full_refresh;

In [0]:
USE IDENTIFIER(catalog_use || "." || schema_use);

In [0]:
SELECT current_catalog(), current_schema();

In [0]:
DECLARE OR REPLACE VARIABLE full_refresh_stmnt STRING; 

SET VARIABLE full_refresh_stmnt = (
  CASE when full_refresh THEN "DROP TABLE IF EXISTS " || resource_type || ";" ELSE "SELECT 'Performing standard refresh of streaming table " || resource_type || "' AS message;" END
);

SELECT full_refresh_stmnt;

In [0]:
EXECUTE IMMEDIATE full_refresh_stmnt;

In [0]:
DECLARE OR REPLACE VARIABLE resource_keys STRING;

SET VAR resource_keys = (
  SELECT 
    array_join(collect_list(DISTINCT resource), "', '")
  FROM 
    resource_schemas
  WHERE 
    resourceType = resource_type
);

select resource_keys;

In [0]:
DECLARE OR REPLACE VARIABLE resource_stmnt STRING;

SET VARIABLE resource_stmnt = "
CREATE OR REFRESH STREAMING TABLE " || resource_type || "
COMMENT 'Parsed FHIR " || resource_type || " Records'
TBLPROPERTIES (
  'delta.enableChangeDataFeed' = 'true'
  ,'delta.enableDeletionVectors' = 'true'
  ,'delta.enableRowTracking' = 'true'
  ,'quality' = 'bronze'
  ,'pipelines.channel' = 'PREVIEW'
  ,'delta.feature.variantType-preview' = 'supported'
)
AS SELECT
  *
FROM (
  SELECT
    bundle_uuid
    ,fullUrl as "|| resource_type ||"_url
    ,key
    ,value
  FROM 
    STREAM(resources_exploded)
  WHERE 
    resourceType = '"|| resource_type ||"')
  PIVOT (
    first(value) FOR key IN ('" || resource_keys || "')
  );
";

SELECT resource_stmnt;


In [0]:
-- SET spark.sql.streaming.stateStore.stateSchemaCheck = false;
SET spark.sql.streaming.stateStore.providerClass = 'com.databricks.sql.streaming.state.RocksDBStateStoreProvider';
EXECUTE IMMEDIATE resource_stmnt;